In [3]:
import pandas as pd

# Loading the directly uploaded files into dataframes
df_accuracy = pd.read_csv("/Users/lijunyi/Downloads/cs506/bus performance/Rapid_Transit_and_Bus_Prediction_Accuracy_Data.csv")
df_routes = pd.read_csv("/Users/lijunyi/Downloads/cs506/bus performance/Bus_Network_Redesign_Draft_Bus_Routes.csv")

# Displaying the first few rows of each dataframe to understand their structure
df_accuracy_head = df_accuracy.head()
df_routes_head = df_routes.head()

df_accuracy_head, df_routes_head


(                   weekly mode route_id        bin arrival_departure  \
 0  2021/08/13 04:00:00+00  bus      NaN    0-3 min         departure   
 1  2021/08/13 04:00:00+00  bus      NaN    3-6 min         departure   
 2  2021/08/13 04:00:00+00  bus      NaN   6-12 min         departure   
 3  2021/08/13 04:00:00+00  bus      NaN  12-30 min         departure   
 4  2021/08/20 04:00:00+00  bus      NaN    0-3 min         departure   
 
    num_predictions  num_accurate_predictions  ObjectId  
 0           293039                    233562         1  
 1           285817                    229090         2  
 2           561098                    472923         3  
 3          1594830                   1405620         4  
 4           285591                    228653         5  ,
    FID  Shape_Leng    shape_id  route_id  agency_id route_shor  \
 0    1    0.175046  1129916754  1ab5a43c          1         90   
 1    2    0.168834  1129916755  1ab5a43c          1         90   
 2    3   

In [4]:
# Loading the unzipped files into dataframes
df_reliability = pd.read_csv("/Users/lijunyi/Downloads/cs506/bus performance/MBTA_Bus%2C_Commuter_Rail%2C_%26_Rapid_Transit_Reliability.csv")
df_arrival_departure = pd.read_csv("/Users/lijunyi/Downloads/cs506/bus performance/MBTA_Bus_Arrival_Departure_Times_2023/MBTA-Bus-Arrival-Departure-Times_2023-06.csv")

# Displaying the first few rows of each dataframe to understand their structure
df_reliability_head = df_reliability.head()
df_arrival_departure_head = df_arrival_departure.head()

df_reliability_head, df_arrival_departure_head


(             service_date gtfs_route_id gtfs_route_short_name  \
 0  2023/09/30 04:00:00+00          Blue                   NaN   
 1  2023/09/30 04:00:00+00           Red                   NaN   
 2  2023/09/30 04:00:00+00       Green-B                     B   
 3  2023/09/30 04:00:00+00       Green-C                     C   
 4  2023/09/30 04:00:00+00       Green-D                     D   
 
   gtfs_route_long_name gtfs_route_desc route_category mode_type  \
 0            Blue Line   Rapid Transit      Blue Line      Rail   
 1             Red Line   Rapid Transit       Red Line      Rail   
 2         Green Line B   Rapid Transit     Green Line      Rail   
 3         Green Line C   Rapid Transit     Green Line      Rail   
 4         Green Line D   Rapid Transit     Green Line      Rail   
 
   peak_offpeak_ind          metric_type  otp_numerator  otp_denominator  \
 0         OFF_PEAK  Passenger Wait Time   40148.279348     40633.521004   
 1         OFF_PEAK  Passenger Wait Time

In [6]:
# Filtering the data to consider only bus routes
df_bus_times = df_arrival_departure[df_arrival_departure['route_id'].str.isnumeric()]

# Extracting the earliest and latest time points for each route and direction
min_times = df_bus_times.groupby(['route_id', 'direction_id']).scheduled.min().reset_index()
max_times = df_bus_times.groupby(['route_id', 'direction_id']).scheduled.max().reset_index()

# Merging the two dataframes to get both min and max times in one dataframe
merged_times = pd.merge(min_times, max_times, on=['route_id', 'direction_id'], suffixes=('_start', '_end'))

# Calculating the end-to-end travel time for each route and direction
merged_times['travel_time'] = pd.to_datetime(merged_times['scheduled_end']) - pd.to_datetime(merged_times['scheduled_start'])

# Displaying the end-to-end travel times for different bus routes
end_to_end_travel_times = merged_times[['route_id', 'direction_id', 'travel_time']]

end_to_end_travel_times.head()

#save to csv
end_to_end_travel_times.to_csv('end_to_end_travel_times.csv', index=False)


The end-to-end travel times for different bus routes (for the month of June 2023) are calculated as shown in the table above. The travel times are represented as the difference between the earliest and latest scheduled times for each route and direction.

For example:

For route "01" in the "Inbound" direction, the end-to-end travel time is approximately 20 hours and 29 minutes.
For route "04" in the "Outbound" direction, the end-to-end travel time is approximately 12 hours and 26 minutes.

I have saved the data in a csv file called "end_to_end_travel_times.csv" 

In [9]:
# Filtering the data to consider only bus routes
df_bus_reliability = df_reliability[df_reliability['mode_type'] == 'Bus']

# Calculating the reliability score for each route
df_bus_reliability['reliability_score'] = (df_bus_reliability['otp_numerator'] / df_bus_reliability['otp_denominator']) * 100

# Averaging the reliability score for each route
average_reliability = df_bus_reliability.groupby('gtfs_route_id').agg({
    'reliability_score': 'mean'
}).reset_index()

# Sorting the routes based on average reliability score
sorted_routes_reliability = average_reliability.sort_values(by='reliability_score').reset_index(drop=True)

sorted_routes_reliability.head()


<ipython-input-9-762926e5dce2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bus_reliability['reliability_score'] = (df_bus_reliability['otp_numerator'] / df_bus_reliability['otp_denominator']) * 100


,gtfs_route_id,reliability_score
0,9703,32.009419
1,449,40.255245
2,448,40.630199
3,459,42.997044
4,747,45.480942


In [10]:
#save to csv
sorted_routes_reliability.to_csv('sorted_routes_reliability.csv', index=False)

Based on the reliability data:

Route "9703" has the lowest average reliability score of approximately 32.01%

Route "CR-Shuttle003" has the highest average reliability score of approximately 92.58%

Thus, from these datas, we can get the disparity between the reliability scores of the different routes. The routes with the lowest reliability scores are the ones with disparity that is the highest.